# Digit Recognizer

**Reference: [Introduction to CNN Keras 0.997(top 6)](https://www.kaggle.com/yassineghouzam/introduction-to-cnn-keras-0-997-top-6)**

In [ ]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

sns.set(style='white', context='notebook', palette='deep')

In [ ]:
#Load data
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
y_train = train['label']
X_train = train.drop(labels = ['label'], axis=1)

del train

In [ ]:
g = sns.countplot(y_train)
y_train.value_counts()

## Check for null and missing values

In [ ]:
X_train.isnull().any().describe()

In [ ]:
test.isnull().any().describe()

## Normalize the data

In [ ]:
# Grayscale normalization to reduce effect of illumination's differences.
X_train = X_train / 255.0
test = test / 255.0

In [ ]:
#reshape image in 3 dimension - height = 28px, width = 28px, canal = 1
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

## Label encoding

In [ ]:
# Encode labels to one hot vectors
y_train = to_categorical(y_train, num_classes=10)

## Split training and validation set

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size =0.1, random_state=2)

In [ ]:
#example
g = plt.imshow(X_train[0][:,:,0])

# CNN

In [ ]:
# Set the CNN model
# [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out

model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='Same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='Same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
#set the optimizer

optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [ ]:
#compile the model

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [ ]:
epochs = 10
batch_size = 86

## Data augmentation


*     Randomly rotate some training images by 10 degrees
*     Randomly Zoom by 10% some training images
*     Randomly shift images horizontally by 10% of the width
*     Randomly shift images vertically by 10% of the height


In [ ]:
datagen = ImageDataGenerator(
            featurewise_center=False, 
            samplewise_center= False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            rotation_range=10,  #randomly rotate image in 0 to 180 degree
            zoom_range=0.1, #randomly zoom image
            width_shift_range=0.1, #randomly shift images horizontally
            height_shift_range=0.1, #randomly shift images vertically
            horizontal_flip = False, #randomly flip images
            vertical_flip = False #randomly flip images
)
datagen.fit(X_train)

In [ ]:
# Fit the model
# epoch = 1
clf = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), epochs=1, validation_data=(X_val, y_val),
                             verbose=2, steps_per_epoch=X_train.shape[0] // batch_size, callbacks=[learning_rate_reduction])

In [ ]:
# Fit the model
clf = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), epochs=epochs, validation_data=(X_val, y_val),
                             verbose=2, steps_per_epoch=X_train.shape[0] // batch_size, callbacks=[learning_rate_reduction])

## Evaluate the model

In [ ]:
# plot the loss and accuracy curves
fig, ax = plt.subplots(2,1)
ax[0].plot(clf.history['loss'], color='b', label='Training loss')
ax[0].plot(clf.history['val_loss'], color='r', label='Validation loss', axes=ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(clf.history['acc'], color='b', label='Training accuracy')
ax[1].plot(clf.history['val_acc'], color='r', label='Validation accuracy')
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
# Look at confusion matrix 

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors 
y_pred_classes = np.argmax(y_pred,axis = 1) 
# Convert validation observations to one hot vectors
y_true = np.argmax(y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(y_true, y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10)) 

In [ ]:
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("digitrecognizer.csv",index=False)